### Summary
개요: 제주지역 부름 서비스지역 확대를 위해 제주 버스 승차객 데이터와 부름 예약 데이터를 비교하여 확대 지역을 선정한다

기간: 2024.04

문제: 부름 서비스지역은 어느정도까지 확대해야 타당한가?
- 확대 지역은 어떻게 선정하는 것이 타당한가?
- 공급 조절은 어떻게 진행하는 것이 타당한가?

해결:
1. 제주 버스 이용 API 데이터를 활용하여 비교 분석해본다   
- 제주 버스 데이터 수집
  - 기간: 2024.03.01 ~ 2024.03.31
    - 버스 이용 data
    - 버스 정류소 data
- 정류소별 승차객 수 비교
- 부름 이용자의 예약/호출 위치와 정류소별 승차객 수 비교

2. 존별 거리 범주 반경내 부름 예약 비중으로 공급 조절
- 존의 거리 범주별 부름 호출 예약 비중 비교
  - 예약 비중 비교
  - 실적 비교(이익률, 건당 이익, 가동률)
    - vs 전체 대비 참조

결과:
1. 승차객이 많은 지역일 수록 주변에서의 부름 예약/호출이 많다



2. 10km이내 부름 호출이 많은 존은 실적도 우수하다
- 배반 비용을 세이브하기 위해 가까운 존으로의 공급을 확대한다
- 전체 가동률 대비 낮은 가동률을 보이는 존의 부름 공급을 축소한다
- 지역내 공급확대: 상예동, 대정읍, 색달동, 강정동, 서귀동
- 거점내 공급확대: 히든클리프 호텔&네이처, 백패커스홈, 호텔브릿지 서귀포

결론:
```
승차객이 많은 지역을 중심으로 서비스 지역을 확대하고
부름 비중이 높은 존 중 실적이 우수한 존 대상으로 공급을 확대한다
```

---
데이터수집: API requests
데이터전처리: 버스 정류소 정보 및 승차객 정보 Merge
- 버스정류소의 stationId와 승차정보의 stationId는 서로 다름
- 버스정류소의 stationName의 중복값처리
- 버스정류소 정보가 잘 못된 결측값 처리   

시각화: Pydeck 라이브러리 활용

---

데이터명세서

In [ ]:
from google.colab import output
output.clear()

### Library Import

In [ ]:
## 라이브러리 임포트
import requests
import pandas as pd

# 데이터 저장
from google.colab import drive
drive.mount('/content/drive')

from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.cloud import bigquery
from oauth2client.client import GoogleCredentials

import gspread
gc = gspread.authorize(creds)

### Data Import

버스 데이터 수집
- 정류소 정보: https://www.jejudatahub.net/mypage/project/view-data/612/myProject
- 이용 정보: https://www.jejudatahub.net/mypage/project/view-data/613/myProject

API를 활용하여 Requests 라이브러리 사용

In [ ]:
# ## 라이브러리 설치
# !pip install requests pandas

In [ ]:
# ## 이용정보 API 요청 및 데이터수집

# # API 설정
# base_url = "https://open.jejudatahub.net/api/proxy/1b1ta1a1ba6t6a1ttt3bD6b1tb1t1D3t"
# project_key = "c7p2jjojtb_tb0t0ct0_2ojb2t3bopor"
# params = {
#     'startDate': '20240301',
#     'endDate': '20240331',
#     'limit': 100  # 페이지 당 최대 컨텐츠 수
# }

# def fetch_data(base_url, project_key, params):
#     all_data = []
#     page = 1
#     has_more = True

#     while has_more:
#         # 현재 페이지 설정
#         current_params = params.copy()
#         current_params['number'] = page

#         # API 요청
#         full_url = f"{base_url}/{project_key}"
#         response = requests.get(full_url, params=current_params)
#         print("Requesting:", response.url)  # 요청 URL 확인
#         print("Status Code:", response.status_code)  # 상태 코드 출력

#         # 응답 내용 확인
#         if response.status_code == 200:
#             try:
#                 data = response.json()
#                 all_data.extend(data['data'])
#                 has_more = data['hasMore']
#                 page += 1
#             except JSONDecodeError:
#                 print("Failed to decode JSON:", response.text)  # JSON 디코드 실패 시 메시지 출력
#                 break
#         else:
#             print("Failed request with status:", response.status_code)
#             print("Response:", response.text)
#             break

#     return all_data

# # 데이터 수집
# data = fetch_data(base_url, project_key, params)

In [ ]:
# # 데이터프레임 생성
# # df = pd.DataFrame(data)
# print(df.head())

In [ ]:
# # 파일 경로 설정
# file_path = '/content/drive/My Drive/bus_data2024.csv'

# # 데이터프레임을 CSV 파일로 저장
# df.to_csv(file_path, index=False)

In [ ]:
# # 버스정류소 정보 API 설정
# bus_stations_url = "https://open.jejudatahub.net/api/proxy/DD11ab6a6t11D16baaa1a2tD26ata161"
# project_key = "c7p2jjojtb_tb0t0ct0_2ojb2t3bopor"
# params = {
#     'limit': 100  # 페이지 당 최대 컨텐츠 수
# }

# def fetch_bus_stations(base_url, project_key, params):
#     all_data = []
#     page = 1
#     has_more = True

#     while has_more:
#         # 현재 페이지 설정
#         current_params = params.copy()
#         current_params['number'] = page

#         # API 요청
#         response = requests.get(f"{base_url}/{project_key}", params=current_params)
#         if response.status_code == 200:
#             data = response.json()
#             all_data.extend(data['data'])
#             has_more = data['hasMore']
#             page += 1
#         else:
#             print("Failed request with status:", response.status_code)
#             break

#     return all_data

# # 데이터 수집
# bus_station_data = fetch_bus_stations(bus_stations_url, project_key, params)

In [ ]:
# # 데이터프레임 생성
# df_bus_stations = pd.DataFrame(bus_station_data)

In [ ]:
# # 파일 경로 설정
# file_path_bus_stations = '/content/drive/My Drive/bus_stations_data2024.csv'

# # 데이터프레임을 CSV 파일로 저장
# df_bus_stations.to_csv(file_path_bus_stations, index=False)

In [ ]:
## 저장된 데이터 불러오기
# 정류소 데이터 저장된 CSV 파일 불러오기
# 저장된 CSV 파일 불러오기
file_path_bus = '/content/drive/My Drive/bus_stations_data2024.csv'

df_bus_stations_loaded = pd.read_csv(file_path_bus)

# 파일 경로 설정
file_path = '/content/drive/My Drive/bus_data2024.csv'

# 저장된 CSV 파일 불러오기
df_loaded = pd.read_csv(file_path)
print(df_loaded.head())  # 파일의 첫 5행 출력

### Data Processing

In [ ]:
# stationName 컬럼 기준으로 중복 제거
df_bus_stations_unique = df_bus_stations_loaded.drop_duplicates(subset='stationName', keep='first')

# 결과 확인
print(df_bus_stations_unique.info())
print(df_bus_stations_unique.head())

In [ ]:
df_loaded['stationName'].unique()

In [ ]:
df_loaded.isnull().sum()

In [ ]:
# df_loaded에서 결측값이 있는 행 제거
df_loaded_cleaned = df_loaded.dropna(subset=['stationName'])

# stationName, moveType, priceType에 따라 userCount를 그룹화하고 합산
df_grouped = df_loaded_cleaned.groupby(['stationName', 'moveType', 'priceType']).agg({'userCount': 'sum'}).reset_index()

# df_bus_stations_loaded에서 필요한 컬럼만 선택
df_bus_stations_subset = df_bus_stations_unique[['stationName', 'stationAddress', 'direction', 'longitude', 'latitude']]

# df_grouped와 df_bus_stations_subset을 stationName을 기준으로 LEFT JOIN 수행
df_merged = pd.merge(df_bus_stations_subset, df_grouped, on='stationName', how='left')

# 결과 출력
print(df_merged.head())

In [ ]:
df_merged.info()

In [ ]:
df_merged.head(3)

In [ ]:
df_merged['priceType'].unique()

In [ ]:
## priceType에서 타겟군 외에는 제외
df_merged_target = df_merged[(df_merged['priceType'] != '어린이') & (df_merged['priceType'] != '청소년')]

In [ ]:
df_merged.shape

In [ ]:
df_merged.isnull().sum()

In [ ]:
## 정류소별 전체 승차인원수와 승차 타입별 인원수 Sum
df_merged_total = df_merged_target.groupby(['stationName', 'stationAddress', 'direction', 'latitude', 'longitude'])['userCount'].sum().reset_index()

# stationName, stationAddress, direction, latitude, longitude, moveType에 따라 userCount를 sum
df_merged_type = df_merged_target.groupby(['stationName', 'stationAddress', 'direction', 'latitude', 'longitude', 'moveType'])['userCount'].sum().reset_index()

### Data Visualization

#### 정류소별 전체 탑승객 수와 승차타입별 시각화 비교
- Pydeck 라이브러리 활용

In [ ]:
!pip install pydeck

In [ ]:
import pydeck as pdk

# priceType에 따른 색상 지정을 위한 함수
def assign_color(row):
    if row['moveType'] == '승차':
        return [0, 0, 255, 255]  # Blue
    elif row['moveType'] == '환승':
        return [255, 0, 0, 255]  # Red
    else:
        return [128, 128, 128, 255]  # Gray for undefined or other types

# 색상 컬럼 추가(priceType)
df_merged_type['color'] = df_merged_type.apply(assign_color, axis=1)

# pydeck을 이용한 시각화를 위한 데이터 준비
def render_map(dataframe, color_scale):
    view_state = pdk.ViewState(latitude=dataframe['latitude'].mean(), longitude=dataframe['longitude'].mean(), zoom=10)
    layer = pdk.Layer(
        "ScatterplotLayer",
        dataframe,
        get_position=['longitude', 'latitude'],
        get_color=color_scale,
        get_radius=100,
        pickable=True
    )
    return pdk.Deck(layers=[layer], initial_view_state=view_state)

def render_map2(dataframe):
    view_state = pdk.ViewState(latitude=dataframe['latitude'].mean(), longitude=dataframe['longitude'].mean(), zoom=10)
    layer = pdk.Layer(
        "ScatterplotLayer",
        dataframe,
        get_position=['longitude', 'latitude'],
        get_color='color',
        get_radius=100,
        pickable=True
    )
    return pdk.Deck(layers=[layer], initial_view_state=view_state)

# 데이터프레임에 따라 색상 지정 (예: 더 많은 사용자가 있는 곳은 더 진한 색)
color_scale_total = ['userCount * 30', 0, 140, 140]

# 지도 렌더링
map_total = render_map(df_merged_total, color_scale_total)
map_type = render_map2(df_merged_type)

In [ ]:
# 전체 승차객수 지도 ()
map_total.show()

In [ ]:
# 승차타입별 지도
## Blue 승차, Red 환승

map_type.show()

In [ ]:
## 승객수별 플랏 크기 키우기
# pydeck을 이용한 시각화를 위한 데이터 준비
def render_map(dataframe, color_scale):
    view_state = pdk.ViewState(latitude=dataframe['latitude'].mean(), longitude=dataframe['longitude'].mean(), zoom=10)
    layer = pdk.Layer(
        "ScatterplotLayer",
        dataframe,
        get_position=['longitude', 'latitude'],
        get_color=color_scale,
        get_radius="userCount",  # 점의 크기를 userCount에 비례하여 조절
        pickable=True,
        opacity=0.6  # 투명도 설정
    )
    return pdk.Deck(layers=[layer], initial_view_state=view_state)

def render_map2(dataframe):
    view_state = pdk.ViewState(latitude=dataframe['latitude'].mean(), longitude=dataframe['longitude'].mean(), zoom=10)
    layer = pdk.Layer(
        "ScatterplotLayer",
        dataframe,
        get_position=['longitude', 'latitude'],
        get_color='color',
        get_radius="userCount",  # 점의 크기를 userCount에 비례하여 조절
        pickable=True,
        opacity=0.6  # 투명도 설정
    )
    return pdk.Deck(layers=[layer], initial_view_state=view_state)

# 데이터프레임에 따라 색상 지정
color_scale_total = ['userCount * 30', 0, 140, 140]

# 지도 렌더링
map_total = render_map(df_merged_total, color_scale_total)
map_type = render_map2(df_merged_type)

In [ ]:
# 전체 승차객 지도 표시
map_total.show()

In [ ]:
# 승차타입별 승차객 지도 표시
map_type.show()

In [ ]:
## 전체 승차객 히트맵
# HeatmapLayer를 사용한 히트맵 시각화(df_merged_total)
heatmap_layer = pdk.Layer(
    'HeatmapLayer',
    df_merged_total,
    get_position=['longitude', 'latitude'],
    get_weight='userCount',
    radius_pixels=60
)

view_state = pdk.ViewState(
    latitude=df_merged_total['latitude'].mean(),
    longitude=df_merged_total['longitude'].mean(),
    zoom=10
)

# Deck 오브젝트 생성 및 렌더링
deck = pdk.Deck(layers=[heatmap_layer], initial_view_state=view_state)
deck.to_html('heatmap.html')

#### 제주 부름 이용자들의 예약 생성/호출 위치와 버스정류소별 승차데이터

In [ ]:
## 기본 데이터 임포트
query = """
  SELECT
      rv.way,
      rv.reservation_id as rid,

      rv.dtod_start_lng as start_lng, -- 호출지
      rv.dtod_start_lat as start_lat, -- 호출
      rv.dtod_end_lng as end_lng,
      rv.dtod_end_lat as end_lat,

      rv.reservation_created_lat as c_lat, -- 생성지
      rv.reservation_created_lng as c_lng, -- 생성지

  FROM `soda_store.reservation_v2` rv
  WHERE date BETWEEN '2024-03-01' AND '2024-03-31'
  AND region1 = '제주특별자치도'
  AND way != 'round'
  AND state IN (1, 2, 3)
  AND dtod_start_lng is not null
  AND dtod_end_lng is not null
  AND reservation_created_lat is not null
  AND reservation_created_lng is not null
  AND rv.member_imaginary IN (0, 9)
  """

df_loc = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
) ## 예약위치 raw

In [ ]:
df_loc.info()

In [ ]:
df_loc.head(5)

In [ ]:
df_loc.shape

In [ ]:
## df_loc에서 예약 생성 위치(c_lat, c_lng)와 호출 위치(start_lat, start_lng)에 대해 각각의 레이어 생성
def create_user_layer(dataframe, lat_col, lng_col, color, radius=100):
    return pdk.Layer(
        "ScatterplotLayer",
        dataframe,
        get_position=[lng_col, lat_col],
        get_color=color,
        get_radius=radius,
        pickable=True,
        opacity=0.6  # 투명도 조정
    )

In [ ]:
## 버스데이터 시각화 함수
def render_complete_map(bus_data, user_data):
    # 기존 버스 정류장 데이터 레이어
    bus_layer = pdk.Layer(
        "ScatterplotLayer",
        bus_data,
        get_position=['longitude', 'latitude'],
        get_color='color',
        get_radius="userCount",
        pickable=True,
        opacity=0.6
    )

    # 사용자 예약 생성 위치 레이어 (Yellow)
    reservation_layer = create_user_layer(user_data, 'c_lat', 'c_lng', [255, 255, 0, 160])

    # 사용자 호출 위치 레이어 (Green)
    call_layer = create_user_layer(user_data, 'start_lat', 'start_lng', [0, 128, 0, 160])

    # 뷰 상태 설정
    view_state = pdk.ViewState(
        latitude=bus_data['latitude'].mean(),
        longitude=bus_data['longitude'].mean(),
        zoom=10
    )

    # 모든 레이어를 포함한 덱 생성
    deck = pdk.Deck(
        layers=[bus_layer, reservation_layer, call_layer],
        initial_view_state=view_state,
    )

    return deck

In [ ]:
# 데이터프레임 로드 및 시각화 실행
final_map = render_complete_map(df_merged_type, df_loc)
final_map.show()

### 존별 부름 실적과 공급 현황

1. 분포: 부름의 예약비중은 분포가 골고루 되어있지 않다
- 15km 미만과 25km 이상에 집중

2. 존 반경 범주내 부름 호출지 비중: 각 범주별 상위존 대상 공급 확대
- 실적이 전체 대비 높다면, 확대
- 비중은 높으나 실적이 전체 대비 낮다면, 축소
  - 3Km 반경내 공급
    - 확대: 히든클리프 호텔&네이처

| zid   | zname                 | GP_ratio | GP_ratio_vs | d2d_3km_rate | d2d_rate | op_rate | op_rate_vs | profit_per_use | profit_use_vs |
|-------|-----------------------|----------|-------------|--------------|----------|---------|------------|----------------|---------------|
| 246   | 켄싱턴리조트 제주중문  | 0.14     | -0.08       | 0.05         | 0.53     | 0.49    | -0.03      | 11074.0        | -0.18         |
| 12812 | 성산 K마트             | 0.20     | -0.02       | 0.04         | 0.14     | 0.59    | 0.06       | 11656.0        | -0.14         |
| 14420 | 서귀포시외버스터미널 맞은편 | 0.20  | -0.02       | 0.05         | 0.33     | 0.51    | -0.02      | 13049.0        | -0.04         |
| 15304 | 북수구광장 옆           | 0.22     | 0.00        | 0.04         | 0.04     | 0.59    | 0.06       | 14221.0        | 0.05          |
| 16126 | 천제연폭포 앞           | 0.33     | 0.11        | 0.04         | 0.16     | 0.54    | 0.02       | 21321.0        | 0.57          |
| 16618 | 롯데 하이마트 서귀포점  | 0.16     | -0.06       | 0.04         | 0.38     | 0.54    | 0.02       | 13231.0        | -0.03         |
| 16620 | 롯데 하이마트 제주점    | 0.16     | -0.07       | 0.05         | 0.05     | 0.54    | 0.01       | 10827.0        | -0.20         |
| 18261 | 히든클리프 호텔&네이쳐   | 0.19     | -0.03       | 0.10         | 0.43     | 0.63    | 0.11       | 15705.0        | 0.16          |
| 18269 | 백패커스홈              | 0.18     | -0.04       | 0.07         | 0.47     | 0.53    | 0.01       | 18635.0        | 0.37          |
| 18728 | 동문로터리              | 0.20     | -0.02       | 0.05         | 0.13     | 0.53    | 0.01       | 11522.0        | -0.15         |


  - 5km 반경내
    - 확대: 백패커스홈
    - 축소: 신신호텔 제주오션

| zid   | zname                 | GP_ratio | GP_ratio_vs | d2d_5km_rate | d2d_rate | op_rate | op_rate_vs | profit_per_use | profit_use_vs |
|-------|-----------------------|----------|-------------|--------------|----------|---------|------------|----------------|---------------|
| 246   | 켄싱턴리조트 제주중문  | 0.14     | -0.08       | 0.05         | 0.53     | 0.49    | -0.03      | 11074.0        | -0.18         |
| 1967  | 신신호텔 제주오션      | 0.10     | -0.12       | 0.07         | 0.33     | 0.51    | -0.01      | 8271.0         | -0.39         |
| 2230  | 서귀포시청 제1청사 부근 | 0.21     | -0.02       | 0.06         | 0.25     | 0.55    | 0.03       | 14563.0        | 0.07          |
| 12762 | 제주호텔 더엠          | 0.18     | -0.04       | 0.11         | 0.37     | 0.52    | 0.00       | 14477.0        | 0.07          |
| 14420 | 서귀포시외버스터미널 맞은편 | 0.20  | -0.02       | 0.07         | 0.33     | 0.51    | -0.02      | 13049.0        | -0.04         |
| 14451 | 호텔브릿지 서귀포      | 0.24     | 0.02        | 0.14         | 0.38     | 0.67    | 0.15       | 20947.0        | 0.54          |
| 17154 | 서귀포 중앙로터리      | 0.20     | -0.02       | 0.11         | 0.23     | 0.55    | 0.03       | 12971.0        | -0.04         |
| 18261 | 히든클리프 호텔&네이쳐   | 0.19     | -0.03       | 0.10         | 0.43     | 0.63    | 0.11       | 15705.0        | 0.16          |
| 18269 | 백패커스홈              | 0.18     | -0.04       | 0.27         | 0.47     | 0.53    | 0.01       | 18635.0        | 0.37          |
| 19018 | 제주항 연안여객터미널(2부두) | 0.24  | 0.01        | 0.06         | 0.06     | 0.53    | 0.01       | 20426.0        | 0.50          |

  - 10km 반경내
    - 확대: 히든클리프 호텔&네이처, 호텔브릿지 서귀포
    - 축소: 신신호텔 제주오션

| zid   | zname                    | GP_ratio | GP_ratio_vs | d2d_10km_rate | d2d_rate | op_rate | op_rate_vs | profit_per_use | profit_use_vs |
|-------|--------------------------|----------|-------------|---------------|----------|---------|------------|----------------|---------------|
| 246   | 켄싱턴리조트 제주중문      | 0.14     | -0.08       | 0.19          | 0.53     | 0.49    | -0.03      | 11074.0        | -0.18         |
| 1967  | 신신호텔 제주오션         | 0.10     | -0.12       | 0.09          | 0.33     | 0.51    | -0.01      | 8271.0         | -0.39         |
| 2230  | 서귀포시청 제1청사 부근    | 0.21     | -0.02       | 0.07          | 0.25     | 0.55    | 0.03       | 14563.0        | 0.07          |
| 12636 | 중문중학교 옆             | 0.22     | -0.01       | 0.13          | 0.21     | 0.46    | -0.06      | 12354.0        | -0.09         |
| 12762 | 제주호텔 더엠             | 0.18     | -0.04       | 0.13          | 0.37     | 0.52    | 0.00       | 14477.0        | 0.07          |
| 14420 | 서귀포시외버스터미널 맞은편 | 0.20     | -0.02       | 0.11          | 0.33     | 0.51    | -0.02      | 13049.0        | -0.04         |
| 14451 | 호텔브릿지 서귀포          | 0.24     | 0.02        | 0.33          | 0.38     | 0.67    | 0.15       | 20947.0        | 0.54          |
| 17154 | 서귀포 중앙로터리          | 0.20     | -0.02       | 0.12          | 0.23     | 0.55    | 0.03       | 12971.0        | -0.04         |
| 18261 | 히든클리프 호텔&네이쳐      | 0.19     | -0.03       | 0.33          | 0.43     | 0.63    | 0.11       | 15705.0        | 0.16          |
| 18269 | 백패커스홈                 | 0.18     | -0.04       | 0.27          | 0.47     | 0.53    | 0.01       | 18635.0        | 0.37          |

3. 지역별 존의 반경 범주내 부름 호출지 비중: 각 범주별 상위 지역 대상 공급 확대
- 실적이 전체 대비 높다면, 확대
- 비중은 높으나 실적이 전체 대비 낮다면, 축소
  - 3Km 반경내
    - 확대: 상예동
    - 축소: 중문동

| region3 | z_cnt | opr_day | d2d_3km_rate | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|--------------|----------|----------|---------|----------------|
| 상예동   | 1     | 1.0     | 0.10         | 0.43     | 0.19     | 0.63    | 15705.0        |
| 강정동   | 2     | 6.0     | 0.05         | 0.33     | 0.20     | 0.51    | 13049.0        |
| 일도일동 | 1     | 2.0     | 0.04         | 0.04     | 0.22     | 0.59    | 14221.0        |
| 삼도이동 | 2     | 3.0     | 0.03         | 0.04     | 0.20     | 0.58    | 14099.0        |
| 색달동   | 3     | 10.0    | 0.03         | 0.36     | 0.18     | 0.47    | 13327.0        |
| 서귀동   | 6     | 14.0    | 0.03         | 0.32     | 0.18     | 0.55    | 14471.0        |
| 동홍동   | 3     | 8.0     | 0.02         | 0.21     | 0.20     | 0.50    | 12083.0        |
| 서호동   | 2     | 5.0     | 0.02         | 0.17     | 0.24     | 0.52    | 13861.0        |
| 성산읍   | 3     | 7.0     | 0.02         | 0.17     | 0.19     | 0.52    | 11655.0        |
| 중문동   | 3     | 9.0     | 0.02         | 0.12     | -0.46    | 0.33    | -3376.0        |

  - 5km 반경

| region3 | z_cnt | opr_day | d2d_5km_rate | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|--------------|----------|----------|---------|----------------|
| 서귀동   | 6     | 14.0    | 0.13         | 0.32     | 0.18     | 0.55    | 14471.0        |
| 상예동   | 1     | 1.0     | 0.10         | 0.43     | 0.19     | 0.63    | 15705.0        |
| 강정동   | 2     | 6.0     | 0.07         | 0.33     | 0.20     | 0.51    | 13049.0        |
| 오라1동  | 1     | 3.0     | 0.04         | 0.04     | 0.25     | 0.61    | 16088.0        |
| 일도일동 | 1     | 2.0     | 0.04         | 0.04     | 0.22     | 0.59    | 14221.0        |
| 건입동   | 2     | 4.0     | 0.03         | 0.03     | 0.26     | 0.54    | 17860.0        |
| 동홍동   | 3     | 8.0     | 0.03         | 0.21     | 0.20     | 0.50    | 12083.0        |
| 삼도이동 | 2     | 3.0     | 0.03         | 0.04     | 0.20     | 0.58    | 14099.0        |
| 색달동   | 3     | 10.0    | 0.03         | 0.36     | 0.18     | 0.47    | 13327.0        |
| 서호동   | 2     | 5.0     | 0.03         | 0.17     | 0.24     | 0.52    | 13861.0        |


  - 10km 반경
    - 확대: 상예동, 강정동, 색달동

| region3 | z_cnt | opr_day | d2d_10km_rate | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|---------------|----------|----------|---------|----------------|
| 상예동   | 1     | 1.0     | 0.33          | 0.43     | 0.19     | 0.63    | 15705.0        |
| 서귀동   | 6     | 14.0    | 0.17          | 0.32     | 0.18     | 0.55    | 14471.0        |
| 강정동   | 2     | 6.0     | 0.11          | 0.33     | 0.20     | 0.51    | 13049.0        |
| 색달동   | 3     | 10.0    | 0.09          | 0.36     | 0.18     | 0.47    | 13327.0        |
| 서호동   | 2     | 5.0     | 0.06          | 0.17     | 0.24     | 0.52    | 13861.0        |
| 중문동   | 3     | 9.0     | 0.06          | 0.12     | -0.46    | 0.33    | -3376.0        |
| 동홍동   | 3     | 8.0     | 0.04          | 0.21     | 0.20     | 0.50    | 12083.0        |
| 오라1동  | 1     | 3.0     | 0.04          | 0.04     | 0.25     | 0.61    | 16088.0        |
| 일도일동 | 1     | 2.0     | 0.04          | 0.04     | 0.22     | 0.59    | 14221.0        |
| 건입동   | 2     | 4.0     | 0.03          | 0.03     | 0.26     | 0.54    | 17860.0        |

  - total
    - 확대: 상예동, 대정읍, 색달동, 강정동, 서귀동

| region3 | z_cnt | opr_day | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|----------|----------|---------|----------------|
| 남원읍   | 1     | 1.0     | 0.63     | -0.05    | 0.28    | -4432.0        |
| 상예동   | 1     | 1.0     | 0.43     | 0.19     | 0.63    | 15705.0        |
| 대정읍   | 1     | 2.0     | 0.38     | 0.19     | 0.47    | 17529.0        |
| 색달동   | 3     | 10.0    | 0.36     | 0.18     | 0.47    | 13327.0        |
| 안덕면   | 4     | 19.0    | 0.35     | 0.19     | 0.56    | 13367.0        |
| 강정동   | 2     | 6.0     | 0.33     | 0.20     | 0.51    | 13049.0        |
| 서귀동   | 6     | 14.0    | 0.32     | 0.18     | 0.55    | 14471.0        |
| 구좌읍   | 1     | 2.0     | 0.27     | 0.16     | 0.41    | 13502.0        |
| 동홍동   | 3     | 8.0     | 0.21     | 0.20     | 0.50    | 12083.0        |
| 서호동   | 2     | 5.0     | 0.17     | 0.24     | 0.52    | 13861.0        |


In [ ]:
## 기본 데이터 임포트
query = """
  SELECT
  r.way as way
  , CASE WHEN r.zone_id IN (105, 9890) THEN 'air'
          WHEN r.zone_id IN (17209, 19175, 19269) THEN 'air_infront'
          ELSE ci.region3 END as region3
  , r.reservation_id as rid,
  st_distance(st_geogpoint(r.zone_lng, r.zone_lat), st_geogpoint(dtod_start_lng, dtod_start_lat)) as rev_distance

  FROM `socar-data.soda_store.reservation_v2` r
  LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
  LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
  LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
  LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
  LEFT JOIN tianjin_replica.carzone_info ci ON r.zone_id = ci.id
  WHERE date BETWEEN '2024-03-25' AND '2024-04-21'
  AND r.member_imaginary IN (0,9)
  AND r.region1 = '제주특별자치도'
  AND r.zone_id NOT IN (105, 9890, 17209, 19175, 19269)
  AND r.way NOT IN ('round', 'z2d_oneway')
  """

df_zone = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
) ## 존 예약위치 raw

df_zone.info()

In [ ]:
df_zone.head()

In [ ]:
df_zone['rev_distance'].describe()

In [ ]:
import plotly.express as px
import scipy.stats as stats
import numpy as np
import plotly.graph_objects as go

In [ ]:
# 히스토그램 생성
fig = px.histogram(df_zone, x='rev_distance', nbins=30, marginal="rug", title="Distribution of rev_distance")

# 데이터의 평균과 표준편차 계산
mu = df_zone['rev_distance'].mean()
sigma = df_zone['rev_distance'].std()

# 정규분포 데이터 생성
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
y = stats.norm.pdf(x, mu, sigma)  # 정규분포 확률밀도함수

# 정규분포 곡선을 히스토그램에 추가
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Normal Distribution'))

# 그래프 표시
fig.show()

In [ ]:
fig = px.histogram(df_zone, x='rev_distance', color='way', nbins=30, marginal="rug", title="Distribution of rev_distance by Way")
fig.show()

In [ ]:
## 존별 특정 거리 범주이내 부름 예약건수의 존 전체 건수 대비 비중
## 전처리 데이터 임포트
query = """
  WITH base AS (
    SELECT
      region3,
      id as zid,
      zone_name as zname,
      lat, lng
    FROM tianjin_replica.carzone_info
    WHERE region1 = '제주특별자치도'
    AND state = 1
  ),

  base_d2d_rev AS (
    SELECT
    r.way as way,
    CASE WHEN r.zone_id IN (105, 9890) THEN 'air'
        WHEN r.zone_id IN (17209, 19175, 19269) THEN 'air_infront'
        ELSE ci.region3 END as region3,
    r.zone_id,
    r.reservation_id as rid,

    dtod_start_lng as s_lng,
    dtod_start_lat as s_lat

    FROM `socar-data.soda_store.reservation_v2` r
    LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
    LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
    LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
    LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
    LEFT JOIN tianjin_replica.carzone_info ci ON r.zone_id = ci.id
    WHERE date BETWEEN '2024-03-25' AND '2024-04-21'
    AND r.member_imaginary IN (0,9)
    AND r.region1 = '제주특별자치도'
    AND r.way NOT IN ('round', 'z2d_oneway')
  ),

  base_d2d AS (
    SELECT
    b.region3,
    b.zid, b.zname,
    count(rid) as rev_d2d_cnt,
    count(CASE WHEN st_distance(st_geogpoint(b.lng, b.lat), st_geogpoint(r.s_lng, r.s_lat)) <= 10000 THEN r.rid END) as rev_10km,
    count(CASE WHEN st_distance(st_geogpoint(b.lng, b.lat), st_geogpoint(r.s_lng, r.s_lat)) <= 5000 THEN r.rid END) as rev_5km,
    count(CASE WHEN st_distance(st_geogpoint(b.lng, b.lat), st_geogpoint(r.s_lng, r.s_lat)) <= 3000 THEN r.rid END) as rev_3km
    FROM base b
    LEFT JOIN base_d2d_rev r ON b.zid = r.zone_id
    GROUP BY region3, zid, zname
  ),

  base_rev AS (
    SELECT
    r.way as way,
    CASE WHEN r.zone_id IN (105, 9890) THEN 'air'
        WHEN r.zone_id IN (17209, 19175, 19269) THEN 'air_infront'
        ELSE ci.region3 END as region3,
    r.zone_id,
    r.reservation_id as rid,

    dtod_start_lng as s_lng,
    dtod_start_lat as s_lat

    FROM `socar-data.soda_store.reservation_v2` r
    LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
    LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
    LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
    LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
    LEFT JOIN tianjin_replica.carzone_info ci ON r.zone_id = ci.id
    WHERE date BETWEEN '2024-03-25' AND '2024-04-21'
    AND r.member_imaginary IN (0,9)
    AND r.region1 = '제주특별자치도'
  ),

  base_tot AS (
    SELECT
    b.region3,
    b.zid, b.zname,

    count(rid) as rev_cnt,
    FROM base b
    LEFT JOIN base_rev r ON b.zid = r.zone_id
    GROUP BY region3, zid, zname
  ),

  base_profit AS (
    SELECT
      p.zone_id as zid,
      p.zone_name as zname,

      count(car_id) as cnt,
      sum(opr_day) as opr_day,
      sum(nuse) as use,
      sum(utime) as dur,
      sum(revenue) as revenue,
      sum(profit) as profit,
      sum(profit)/sum(revenue) as GP_ratio,
      sum(profit)/sum(nuse) as profit_per_use,
      sum(utime)/(sum(opr_day)*24) as op_rate


    FROM `socar_biz_profit.profit_socar_car_daily` p
    LEFT JOIN tianjin_replica.carzone_info c ON p.zone_id = c.id
    WHERE p.region1 in ('제주특별자치도')
    AND zone_id not in(122,2184,12072,12073,10736,10738,11947,11480,13228,13787,13858,14494,14528,14541,14542)
    AND date BETWEEN '2024-03-25' AND '2024-04-21'
    AND car_sharing_type in ('socar','zplus')
    GROUP BY zid, zname
  ),

  base_profit_total AS (
    SELECT
      9999 as zid,
      '전체' as zname,

      count(car_id) as cnt,
      sum(opr_day) as opr_day,
      sum(nuse) as use,
      sum(utime) as dur,
      sum(revenue) as revenue,
      sum(profit) as profit,
      sum(profit)/sum(revenue) as total_GP_ratio,
      sum(profit)/sum(nuse) as total_profit_per_use,
      sum(utime)/(sum(opr_day)*24) as total_op_rate


    FROM `socar_biz_profit.profit_socar_car_daily` p
    LEFT JOIN tianjin_replica.carzone_info c ON p.zone_id = c.id
    WHERE p.region1 in ('제주특별자치도')
    AND zone_id not in(122,2184,12072,12073,10736,10738,11947,11480,13228,13787,13858,14494,14528,14541,14542,105,9890,17209,19175, 19269)
    AND date BETWEEN '2024-03-25' AND '2024-04-21'
    AND car_sharing_type in ('socar','zplus')
    GROUP BY zid, zname
  ),

  profit_total AS (
    SELECT
      b.zid, b.zname,
      round(b.GP_ratio, 2) as GP_ratio,
      round(b.GP_ratio - bt.total_GP_ratio, 2) as GP_ratio_vs,
      round(b.profit_per_use, 0) as profit_per_use,
      round((b.profit_per_use / bt.total_profit_per_use) -1, 2) as profit_use_vs,
      round(b.op_rate, 2) as op_rate,
      round(b.op_rate - bt.total_op_rate, 2) as op_rate_vs
    FROM base_profit b
    CROSS JOIN base_profit_total bt
  )

  SELECT
    b.*,
    bt.rev_cnt,
    round(safe_divide(b2.rev_3km, bt.rev_cnt), 2) as d2d_3km_rate,
    round(safe_divide(b2.rev_5km, bt.rev_cnt), 2) as d2d_5km_rate,
    round(safe_divide(b2.rev_10km, bt.rev_cnt), 2) as d2d_10km_rate,
    round(safe_divide(b2.rev_d2d_cnt, bt.rev_cnt), 2) as d2d_rate,

    GP_ratio,
    GP_ratio_vs, -- 전체 대비 손익률
    profit_per_use,
    profit_use_vs, -- 전체 대비 건당 이익
    op_rate,
    op_rate_vs -- 전체 대비 가동률 차이 (%p)
  FROM base b
  LEFT JOIN base_d2d b2 ON b.zid = b2.zid
  LEFT JOIN base_tot bt ON b.zid = bt.zid
  LEFT JOIN profit_total bp ON b.zid = bp.zid
  ORDER BY zid
  """

df_rev_info = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
) ## 존 예약위치 raw

df_rev_info.info()

In [ ]:
# 데이터 필터링 및 피벗 테이블 생성
# 각 거점의 부름 예약 비중별로 상위 10개 거점을 필터링하고, 이를 피벗 테이블로 요약

# 상위 10개 거점 필터링 함수
def filter_top_zones(df, rate_column):
    return df.nlargest(10, rate_column)

# 각 거점 별 비율에 따른 상위 10개 거점 필터링
top_3km = filter_top_zones(df_rev_info, 'd2d_3km_rate')
top_5km = filter_top_zones(df_rev_info, 'd2d_5km_rate')
top_10km = filter_top_zones(df_rev_info, 'd2d_10km_rate')
top_overall = filter_top_zones(df_rev_info, 'd2d_rate')

# 실적 피벗 테이블 생성 함수
def create_pivot_table(df):
    return df.pivot_table(index=['zid', 'zname'], values=['d2d_3km_rate', 'd2d_5km_rate', 'd2d_10km_rate', 'd2d_rate', 'GP_ratio', 'GP_ratio_vs', 'profit_per_use', 'profit_use_vs', 'op_rate', 'op_rate_vs'])

# 각 비율별 피벗 테이블 생성
pivot_3km = create_pivot_table(top_3km)
pivot_5km = create_pivot_table(top_5km)
pivot_10km = create_pivot_table(top_10km)
pivot_overall = create_pivot_table(top_overall)

# GP_ratio 시각화
def plot_performance(df, value):
    fig = px.bar(df, x='zname', y=value, color=value, title=f'{value} by Zone')
    fig.show()

# Pydeck 지도 시각화 함수
def render_map(df):
    view_state = pdk.ViewState(latitude=df['lat'].mean(), longitude=df['lng'].mean(), zoom=8)

    layer = pdk.Layer(
        "ScatterplotLayer",
        df,
        get_position=['lng', 'lat'],
        get_color='[200, 30, 0, 160]',
        get_radius='op_rate * 1000',  # op_rate에 따라 크기 조절
        pickable=True,
        opacity=0.8
    )

    deck = pdk.Deck(layers=[layer], initial_view_state=view_state)
    return deck

In [ ]:
# 예시 출력
print(pivot_3km)

| zid   | zname                 | GP_ratio | GP_ratio_vs | d2d_3km_rate | d2d_rate | op_rate | op_rate_vs | profit_per_use | profit_use_vs |
|-------|-----------------------|----------|-------------|--------------|----------|---------|------------|----------------|---------------|
| 246   | 켄싱턴리조트 제주중문  | 0.14     | -0.08       | 0.05         | 0.53     | 0.49    | -0.03      | 11074.0        | -0.18         |
| 12812 | 성산 K마트             | 0.20     | -0.02       | 0.04         | 0.14     | 0.59    | 0.06       | 11656.0        | -0.14         |
| 14420 | 서귀포시외버스터미널 맞은편 | 0.20  | -0.02       | 0.05         | 0.33     | 0.51    | -0.02      | 13049.0        | -0.04         |
| 15304 | 북수구광장 옆           | 0.22     | 0.00        | 0.04         | 0.04     | 0.59    | 0.06       | 14221.0        | 0.05          |
| 16126 | 천제연폭포 앞           | 0.33     | 0.11        | 0.04         | 0.16     | 0.54    | 0.02       | 21321.0        | 0.57          |
| 16618 | 롯데 하이마트 서귀포점  | 0.16     | -0.06       | 0.04         | 0.38     | 0.54    | 0.02       | 13231.0        | -0.03         |
| 16620 | 롯데 하이마트 제주점    | 0.16     | -0.07       | 0.05         | 0.05     | 0.54    | 0.01       | 10827.0        | -0.20         |
| 18261 | 히든클리프 호텔&네이쳐   | 0.19     | -0.03       | 0.10         | 0.43     | 0.63    | 0.11       | 15705.0        | 0.16          |
| 18269 | 백패커스홈              | 0.18     | -0.04       | 0.07         | 0.47     | 0.53    | 0.01       | 18635.0        | 0.37          |
| 18728 | 동문로터리              | 0.20     | -0.02       | 0.05         | 0.13     | 0.53    | 0.01       | 11522.0        | -0.15         |


In [ ]:
plot_performance(top_3km, 'GP_ratio')

In [ ]:
plot_performance(top_3km, 'op_rate')

In [ ]:
# 지도 렌더링
map_deck = render_map(top_3km)
map_deck.to_html('top_3km_map.html')

In [ ]:
print(pivot_5km)

| zid   | zname                 | GP_ratio | GP_ratio_vs | d2d_5km_rate | d2d_rate | op_rate | op_rate_vs | profit_per_use | profit_use_vs |
|-------|-----------------------|----------|-------------|--------------|----------|---------|------------|----------------|---------------|
| 246   | 켄싱턴리조트 제주중문  | 0.14     | -0.08       | 0.05         | 0.53     | 0.49    | -0.03      | 11074.0        | -0.18         |
| 1967  | 신신호텔 제주오션      | 0.10     | -0.12       | 0.07         | 0.33     | 0.51    | -0.01      | 8271.0         | -0.39         |
| 2230  | 서귀포시청 제1청사 부근 | 0.21     | -0.02       | 0.06         | 0.25     | 0.55    | 0.03       | 14563.0        | 0.07          |
| 12762 | 제주호텔 더엠          | 0.18     | -0.04       | 0.11         | 0.37     | 0.52    | 0.00       | 14477.0        | 0.07          |
| 14420 | 서귀포시외버스터미널 맞은편 | 0.20  | -0.02       | 0.07         | 0.33     | 0.51    | -0.02      | 13049.0        | -0.04         |
| 14451 | 호텔브릿지 서귀포      | 0.24     | 0.02        | 0.14         | 0.38     | 0.67    | 0.15       | 20947.0        | 0.54          |
| 17154 | 서귀포 중앙로터리      | 0.20     | -0.02       | 0.11         | 0.23     | 0.55    | 0.03       | 12971.0        | -0.04         |
| 18261 | 히든클리프 호텔&네이쳐   | 0.19     | -0.03       | 0.10         | 0.43     | 0.63    | 0.11       | 15705.0        | 0.16          |
| 18269 | 백패커스홈              | 0.18     | -0.04       | 0.27         | 0.47     | 0.53    | 0.01       | 18635.0        | 0.37          |
| 19018 | 제주항 연안여객터미널(2부두) | 0.24  | 0.01        | 0.06         | 0.06     | 0.53    | 0.01       | 20426.0        | 0.50          |


In [ ]:
plot_performance(top_5km, 'GP_ratio')

In [ ]:
plot_performance(top_5km, 'op_rate')

In [ ]:
# 지도 렌더링
map_deck = render_map(top_5km)
map_deck.to_html('top_5km_map.html')

In [ ]:
print(pivot_10km)

| zid   | zname                    | GP_ratio | GP_ratio_vs | d2d_10km_rate | d2d_rate | op_rate | op_rate_vs | profit_per_use | profit_use_vs |
|-------|--------------------------|----------|-------------|---------------|----------|---------|------------|----------------|---------------|
| 246   | 켄싱턴리조트 제주중문      | 0.14     | -0.08       | 0.19          | 0.53     | 0.49    | -0.03      | 11074.0        | -0.18         |
| 1967  | 신신호텔 제주오션         | 0.10     | -0.12       | 0.09          | 0.33     | 0.51    | -0.01      | 8271.0         | -0.39         |
| 2230  | 서귀포시청 제1청사 부근    | 0.21     | -0.02       | 0.07          | 0.25     | 0.55    | 0.03       | 14563.0        | 0.07          |
| 12636 | 중문중학교 옆             | 0.22     | -0.01       | 0.13          | 0.21     | 0.46    | -0.06      | 12354.0        | -0.09         |
| 12762 | 제주호텔 더엠             | 0.18     | -0.04       | 0.13          | 0.37     | 0.52    | 0.00       | 14477.0        | 0.07          |
| 14420 | 서귀포시외버스터미널 맞은편 | 0.20     | -0.02       | 0.11          | 0.33     | 0.51    | -0.02      | 13049.0        | -0.04         |
| 14451 | 호텔브릿지 서귀포          | 0.24     | 0.02        | 0.33          | 0.38     | 0.67    | 0.15       | 20947.0        | 0.54          |
| 17154 | 서귀포 중앙로터리          | 0.20     | -0.02       | 0.12          | 0.23     | 0.55    | 0.03       | 12971.0        | -0.04         |
| 18261 | 히든클리프 호텔&네이쳐      | 0.19     | -0.03       | 0.33          | 0.43     | 0.63    | 0.11       | 15705.0        | 0.16          |
| 18269 | 백패커스홈                 | 0.18     | -0.04       | 0.27          | 0.47     | 0.53    | 0.01       | 18635.0        | 0.37          |


In [ ]:
plot_performance(top_10km, 'GP_ratio')

In [ ]:
plot_performance(top_10km, 'op_rate')

In [ ]:
# 지도 렌더링
map_deck = render_map(top_10km)
map_deck.to_html('top_10km_map.html')

In [ ]:
## region3별 특정 거리 범주이내 부름 예약건수의 존 전체 건수 대비 비중
## 전처리 데이터 임포트
query = """
  WITH base AS (
    SELECT
      region3,
      id as zid,
      zone_name as zname,
      lat, lng
    FROM tianjin_replica.carzone_info
    WHERE region1 = '제주특별자치도'
    AND state = 1
  ),

  base_d2d_rev AS (
    SELECT
    r.way as way,
    CASE WHEN r.zone_id IN (105, 9890) THEN 'air'
        WHEN r.zone_id IN (17209, 19175, 19269) THEN 'air_infront'
        ELSE ci.region3 END as region3,
    r.zone_id,
    r.reservation_id as rid,

    dtod_start_lng as s_lng,
    dtod_start_lat as s_lat

    FROM `socar-data.soda_store.reservation_v2` r
    LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
    LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
    LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
    LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
    LEFT JOIN tianjin_replica.carzone_info ci ON r.zone_id = ci.id
    WHERE date BETWEEN '2024-03-25' AND '2024-04-21'
    AND r.member_imaginary IN (0,9)
    AND r.region1 = '제주특별자치도'
    AND r.way NOT IN ('round', 'z2d_oneway')
  ),

  base_d2d AS (
    SELECT
    b.region3,
    b.zid, b.zname,
    count(rid) as rev_d2d_cnt,
    count(CASE WHEN st_distance(st_geogpoint(b.lng, b.lat), st_geogpoint(r.s_lng, r.s_lat)) <= 10000 THEN r.rid END) as rev_10km,
    count(CASE WHEN st_distance(st_geogpoint(b.lng, b.lat), st_geogpoint(r.s_lng, r.s_lat)) <= 5000 THEN r.rid END) as rev_5km,
    count(CASE WHEN st_distance(st_geogpoint(b.lng, b.lat), st_geogpoint(r.s_lng, r.s_lat)) <= 3000 THEN r.rid END) as rev_3km
    FROM base b
    LEFT JOIN base_d2d_rev r ON b.zid = r.zone_id
    GROUP BY region3, zid, zname
  ),

  base_rev AS (
    SELECT
    r.way as way,
    CASE WHEN r.zone_id IN (105, 9890) THEN 'air'
        WHEN r.zone_id IN (17209, 19175, 19269) THEN 'air_infront'
        ELSE ci.region3 END as region3,
    r.zone_id,
    r.reservation_id as rid,

    dtod_start_lng as s_lng,
    dtod_start_lat as s_lat

    FROM `socar-data.soda_store.reservation_v2` r
    LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
    LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
    LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
    LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
    LEFT JOIN tianjin_replica.carzone_info ci ON r.zone_id = ci.id
    WHERE date BETWEEN '2024-03-25' AND '2024-04-21'
    AND r.member_imaginary IN (0,9)
    AND r.region1 = '제주특별자치도'
  ),

  base_tot AS (
    SELECT
    b.region3,
    b.zid, b.zname,

    count(rid) as rev_cnt,
    FROM base b
    LEFT JOIN base_rev r ON b.zid = r.zone_id
    GROUP BY region3, zid, zname
  ),

  base_profit AS (
    SELECT
      p.zone_id as zid,
      p.zone_name as zname,

      count(car_id) as cnt,
      sum(opr_day) as opr_day,
      sum(nuse) as use,
      sum(utime) as dur,
      sum(revenue) as revenue,
      sum(profit) as profit,
      sum(profit)/sum(revenue) as GP_ratio,
      sum(profit)/sum(nuse) as profit_per_use,
      sum(utime)/(sum(opr_day)*24) as op_rate


    FROM `socar_biz_profit.profit_socar_car_daily` p
    LEFT JOIN tianjin_replica.carzone_info c ON p.zone_id = c.id
    WHERE p.region1 in ('제주특별자치도')
    AND zone_id not in(122,2184,12072,12073,10736,10738,11947,11480,13228,13787,13858,14494,14528,14541,14542)
    AND date BETWEEN '2024-03-25' AND '2024-04-21'
    AND car_sharing_type in ('socar','zplus')
    GROUP BY zid, zname
  ),

  base_profit_total AS (
    SELECT
      9999 as zid,
      '전체' as zname,

      count(car_id) as cnt,
      sum(opr_day) as opr_day,
      sum(nuse) as use,
      sum(utime) as dur,
      sum(revenue) as revenue,
      sum(profit) as profit,
      sum(profit)/sum(revenue) as total_GP_ratio,
      sum(profit)/sum(nuse) as total_profit_per_use,
      sum(utime)/(sum(opr_day)*24) as total_op_rate


    FROM `socar_biz_profit.profit_socar_car_daily` p
    LEFT JOIN tianjin_replica.carzone_info c ON p.zone_id = c.id
    WHERE p.region1 in ('제주특별자치도')
    AND zone_id not in(122,2184,12072,12073,10736,10738,11947,11480,13228,13787,13858,14494,14528,14541,14542,105,9890,17209,19175, 19269)
    AND date BETWEEN '2024-03-25' AND '2024-04-21'
    AND car_sharing_type in ('socar','zplus')
    GROUP BY zid, zname
  ),

  profit_total AS (
    SELECT
      b.zid, b.zname,
      b.opr_day as opr_day,
      round(b.GP_ratio, 2) as GP_ratio,
      round(b.GP_ratio - bt.total_GP_ratio, 2) as GP_ratio_vs,
      round(b.profit_per_use, 0) as profit_per_use,
      round((b.profit_per_use / bt.total_profit_per_use) -1, 2) as profit_use_vs,
      round(b.op_rate, 2) as op_rate,
      round(b.op_rate - bt.total_op_rate, 2) as op_rate_vs
    FROM base_profit b
    CROSS JOIN base_profit_total bt
  )

  SELECT
    region3,
    count(zid) as z_cnt,
    round(sum(opr_day), 0) as opr_day,
    round(avg(d2d_3km_rate), 2) as d2d_3km_rate,
    round(avg(d2d_5km_rate), 2) as d2d_5km_rate,
    round(avg(d2d_10km_rate), 2) as d2d_10km_rate,
    round(avg(d2d_rate), 2) as d2d_rate,
    round(avg(GP_ratio), 2) as GP_ratio,
    round(avg(op_rate), 2) as op_rate,
    round(avg(profit_per_use), 0) as profit_per_use,

  FROM (
        SELECT
          b.*,
          bt.rev_cnt,
          b2.rev_3km,
          round(safe_divide(b2.rev_3km, bt.rev_cnt), 2) as d2d_3km_rate,
          round(safe_divide(b2.rev_5km, bt.rev_cnt), 2) as d2d_5km_rate,
          round(safe_divide(b2.rev_10km, bt.rev_cnt), 2) as d2d_10km_rate,
          round(safe_divide(b2.rev_d2d_cnt, bt.rev_cnt), 2) as d2d_rate,
          opr_day/28 as opr_day,

          GP_ratio,
          GP_ratio_vs, -- 전체 대비 손익률
          profit_per_use,
          profit_use_vs, -- 전체 대비 건당 이익
          op_rate,
          op_rate_vs -- 전체 대비 가동률 차이 (%p)
        FROM base b
        LEFT JOIN base_d2d b2 ON b.zid = b2.zid
        LEFT JOIN base_tot bt ON b.zid = bt.zid
        LEFT JOIN profit_total bp ON b.zid = bp.zid
  )
  GROUP BY region3
  ORDER BY region3
  """

df_region3 = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
) ## 존 예약위치 raw

df_region3.info()

In [ ]:
# 데이터 필터링 및 피벗 테이블 생성

# 상위 10개 지역 필터링 함수
def filter_top_zones(df, rate_column):
    return df.nlargest(10, rate_column)

# 각 거점 별 비율에 따른 상위 10개 지역 필터링
top_3km = filter_top_zones(df_region3, 'd2d_3km_rate')
top_5km = filter_top_zones(df_region3, 'd2d_5km_rate')
top_10km = filter_top_zones(df_region3, 'd2d_10km_rate')
top_overall = filter_top_zones(df_region3, 'd2d_rate')

# 실적 피벗 테이블 생성 함수
def create_pivot_table(df):
    return df.pivot_table(index='region3', values=['d2d_3km_rate', 'd2d_5km_rate', 'd2d_10km_rate', 'd2d_rate', 'GP_ratio', 'profit_per_use', 'op_rate'])

# 각 비율별 피벗 테이블 생성
pivot_3km = create_pivot_table(top_3km)
pivot_5km = create_pivot_table(top_5km)
pivot_10km = create_pivot_table(top_10km)
pivot_overall = create_pivot_table(top_overall)

# GP_ratio 시각화
def plot_performance(df, value):
    fig = px.bar(df, x='region3', y=value, color=value, title=f'{value} by Region3')
    fig.show()

In [ ]:
print(top_3km)

| region3 | z_cnt | opr_day | d2d_3km_rate | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|--------------|----------|----------|---------|----------------|
| 상예동   | 1     | 1.0     | 0.10         | 0.43     | 0.19     | 0.63    | 15705.0        |
| 강정동   | 2     | 6.0     | 0.05         | 0.33     | 0.20     | 0.51    | 13049.0        |
| 일도일동 | 1     | 2.0     | 0.04         | 0.04     | 0.22     | 0.59    | 14221.0        |
| 삼도이동 | 2     | 3.0     | 0.03         | 0.04     | 0.20     | 0.58    | 14099.0        |
| 색달동   | 3     | 10.0    | 0.03         | 0.36     | 0.18     | 0.47    | 13327.0        |
| 서귀동   | 6     | 14.0    | 0.03         | 0.32     | 0.18     | 0.55    | 14471.0        |
| 동홍동   | 3     | 8.0     | 0.02         | 0.21     | 0.20     | 0.50    | 12083.0        |
| 서호동   | 2     | 5.0     | 0.02         | 0.17     | 0.24     | 0.52    | 13861.0        |
| 성산읍   | 3     | 7.0     | 0.02         | 0.17     | 0.19     | 0.52    | 11655.0        |
| 중문동   | 3     | 9.0     | 0.02         | 0.12     | -0.46    | 0.33    | -3376.0        |


In [ ]:
print(top_5km)

| region3 | z_cnt | opr_day | d2d_5km_rate | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|--------------|----------|----------|---------|----------------|
| 서귀동   | 6     | 14.0    | 0.13         | 0.32     | 0.18     | 0.55    | 14471.0        |
| 상예동   | 1     | 1.0     | 0.10         | 0.43     | 0.19     | 0.63    | 15705.0        |
| 강정동   | 2     | 6.0     | 0.07         | 0.33     | 0.20     | 0.51    | 13049.0        |
| 오라1동  | 1     | 3.0     | 0.04         | 0.04     | 0.25     | 0.61    | 16088.0        |
| 일도일동 | 1     | 2.0     | 0.04         | 0.04     | 0.22     | 0.59    | 14221.0        |
| 건입동   | 2     | 4.0     | 0.03         | 0.03     | 0.26     | 0.54    | 17860.0        |
| 동홍동   | 3     | 8.0     | 0.03         | 0.21     | 0.20     | 0.50    | 12083.0        |
| 삼도이동 | 2     | 3.0     | 0.03         | 0.04     | 0.20     | 0.58    | 14099.0        |
| 색달동   | 3     | 10.0    | 0.03         | 0.36     | 0.18     | 0.47    | 13327.0        |
| 서호동   | 2     | 5.0     | 0.03         | 0.17     | 0.24     | 0.52    | 13861.0        |


In [ ]:
print(top_10km)

| region3 | z_cnt | opr_day | d2d_10km_rate | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|---------------|----------|----------|---------|----------------|
| 상예동   | 1     | 1.0     | 0.33          | 0.43     | 0.19     | 0.63    | 15705.0        |
| 서귀동   | 6     | 14.0    | 0.17          | 0.32     | 0.18     | 0.55    | 14471.0        |
| 강정동   | 2     | 6.0     | 0.11          | 0.33     | 0.20     | 0.51    | 13049.0        |
| 색달동   | 3     | 10.0    | 0.09          | 0.36     | 0.18     | 0.47    | 13327.0        |
| 서호동   | 2     | 5.0     | 0.06          | 0.17     | 0.24     | 0.52    | 13861.0        |
| 중문동   | 3     | 9.0     | 0.06          | 0.12     | -0.46    | 0.33    | -3376.0        |
| 동홍동   | 3     | 8.0     | 0.04          | 0.21     | 0.20     | 0.50    | 12083.0        |
| 오라1동  | 1     | 3.0     | 0.04          | 0.04     | 0.25     | 0.61    | 16088.0        |
| 일도일동 | 1     | 2.0     | 0.04          | 0.04     | 0.22     | 0.59    | 14221.0        |
| 건입동   | 2     | 4.0     | 0.03          | 0.03     | 0.26     | 0.54    | 17860.0        |


In [ ]:
print(top_overall)

| region3 | z_cnt | opr_day | d2d_rate | GP_ratio | op_rate | profit_per_use |
|---------|-------|---------|----------|----------|---------|----------------|
| 남원읍   | 1     | 1.0     | 0.63     | -0.05    | 0.28    | -4432.0        |
| 상예동   | 1     | 1.0     | 0.43     | 0.19     | 0.63    | 15705.0        |
| 대정읍   | 1     | 2.0     | 0.38     | 0.19     | 0.47    | 17529.0        |
| 색달동   | 3     | 10.0    | 0.36     | 0.18     | 0.47    | 13327.0        |
| 안덕면   | 4     | 19.0    | 0.35     | 0.19     | 0.56    | 13367.0        |
| 강정동   | 2     | 6.0     | 0.33     | 0.20     | 0.51    | 13049.0        |
| 서귀동   | 6     | 14.0    | 0.32     | 0.18     | 0.55    | 14471.0        |
| 구좌읍   | 1     | 2.0     | 0.27     | 0.16     | 0.41    | 13502.0        |
| 동홍동   | 3     | 8.0     | 0.21     | 0.20     | 0.50    | 12083.0        |
| 서호동   | 2     | 5.0     | 0.17     | 0.24     | 0.52    | 13861.0        |


In [ ]:
# 시각화 실행
plot_performance(top_3km, 'GP_ratio')
plot_performance(top_5km, 'GP_ratio')
plot_performance(top_10km, 'GP_ratio')
plot_performance(top_overall, 'GP_ratio')

In [ ]:
# 시각화 실행
plot_performance(top_3km, 'op_rate')
plot_performance(top_5km, 'op_rate')
plot_performance(top_10km, 'op_rate')
plot_performance(top_overall, 'op_rate')